In [ ]:
#wan
model_train_list = {
  "clip_l": "https://huggingface.co/StableDiffusionVN/Flux/blob/main/Clip/clip_l.safetensors",
  "t5_xxl": "https://huggingface.co/StableDiffusionVN/Flux/blob/main/Clip/t5xxl_fp16.safetensors",
  "flux_vae": "https://huggingface.co/StableDiffusionVN/Flux/blob/main/Vae/flux_vae.safetensors",

  "kontext_dev": "https://huggingface.co/StableDiffusionVN/Flux/blob/main/Unet/flux1-kontext-dev.safetensors",
  "qwen_image_fp8": "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/blob/main/split_files/diffusion_models/qwen_image_fp8_e4m3fn.safetensors",
  "clip_qwen": "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/blob/main/split_files/text_encoders/qwen_2.5_vl_7b.safetensors",
  "vae_qwen": "https://huggingface.co/StableDiffusionVN/QwenImage/blob/main/vae/qwen_vae.safetensors"
}

#model_dic

model_dic = {
    "kontext_dev": {
        "model": "kontext_dev",
        "vae": "flux_vae",
        "clip": "t5_xxl",
        "clip2": "clip_l",
    },
    "qwen_image": {
        "model": "qwen_image_fp8",
        "vae": "vae_qwen",
        "clip": "clip_qwen",
    }

}

model_folder = os.path.join(root_dir,"models")

In [ ]:
def dic2arg(config:dict):
  arg = ''
  for value in config:
    arg += f'{value if str(config[value]) != "False" else ""} {"" if type(config[value]) == bool else config[value]} '
  return arg

def hug_down(link,path):
  name = path.split('/')[-1]
  folder = path.split(name)[0]
  if "blob" in link:
    link = link.replace("blob","resolve")
  !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -s 16 -k 1M {link} -d {folder} -o {name}

def model_path(model):
  model_name = model
  model_name = model_train_list[model].split('/')[-1]
  model_path = os.path.join(model_folder,model_name)
  if not os.path.exists(model_path):
    hug_down(model_train_list[model],model_path)
  return model_path

op = {
    '--mixed_precision': 'bf16',
    '--num_cpu_threads_per_process': 1,
}

def data_config():
  return {
    "general": {
        "resolution": resolution,
        "caption_extension": extension,
        "batch_size": batch_size,
        "enable_bucket": True,
        "bucket_no_upscale": False
    },
    "datasets": []
  }

def config():
  dataset_config_path = os.path.join(root_dir,"dataset.toml")
  return {
    "flux_kontext_train_network.py": True if "kontext" in model_type else False,
    "qwen_image_train_network.py": True if "qwen" in model_type else False,
    "--dit": f'"{model_path(model_dic[model_type]["model"])}"' if model_train == "" else download_lib(model_train),
    "--vae": f'"{model_path(model_dic[model_type]["vae"])}"',
    "--text_encoder": f'"{model_path(model_dic[model_type]["clip"])}"' if "clip2" not in model_dic[model_type] else False,
    "--text_encoder1": f'"{model_path(model_dic[model_type]["clip"])}"' if "clip2" in model_dic[model_type] else False,
    "--text_encoder2": f'"{model_path(model_dic[model_type]["clip2"])}"' if "clip2" in model_dic[model_type] else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--sdpa": True,
    "--flash_attn": False,
    "--flash3": False,
    "--sage_attn": False,
    "--xformers": False,
    "--split_attn": True,
    "--mixed_precision": "bf16",
    "--fp8_base": True,
    "--optimizer_type": optimizer_type,
    "--optimizer_args": False,
    "--learning_rate": learning_rate,
    "--lr_scheduler": lr_scheduler,
    "--gradient_checkpointing": True,
    "--gradient_accumulation_steps": False,
    "--max_data_loader_n_workers": 2,
    "--persistent_data_loader_workers": True,
    "--network_module": "networks.lora_flux" if "kontext" in model_type else "networks.lora_qwen_image",
    "--network_dim": network_dim,
    "--network_alpha": network_alpha,
    "--network_dropout": False,
    "--network_weights": False,
    "--blocks_to_swap": 16 if blocks_to_swap and "qwen" in model_type else False,
    "--timestep_sampling": "flux_shift" if "kontext" in model_type else "qwen_shift",
    "--discrete_flow_shift": 3.0 if "qwen" in model_type else False,
    "--weighting_scheme": "none",
    "--max_train_steps": False,
    "--max_train_epochs": max_train_epochs,
    "--save_every_n_epochs": save_every_n_epochs,
    "--save_last_n_epochs": False if save_last_n_epochs <= 0 else save_last_n_epochs,
    "--save_every_n_steps": False if save_every_n_steps <= 0 else save_every_n_steps,
    "--seed": 42,
    "--output_dir": f'"{output_dir}/{output_name}_{model_type}"',
    "--output_name": f'"{output_name}"'
}

sample_prompt_path = os.path.join(root_dir,"prompt.txt")

def extra():
  return {
    "--sample_prompts": f'"{sample_prompt_path}"' if sample_every_n_steps > 0 else False,
    "--sample_every_n_epochs": False,
    "--sample_every_n_steps": sample_every_n_steps if sample_every_n_steps > 0 else False,
    "--sample_at_first": True if sample_every_n_steps > 0 else False,
    "--log_with": "wandb" if wandb_api_key != "" else False,
    "--wandb_api_key": wandb_api_key if wandb_api_key != "" else False,
    "--wandb_run_name": f'"{model_type}_{output_name}"',
    "--save_state": save_state,
    "--save_state_on_train_end": save_state_on_train_end,
    "--resume": resume if resume != "" else False,
    '--metadata_title': f'"{output_name}"',
    '--metadata_author': f'"{author}"',
    '--metadata_description': f'"{description}"',
    '--metadata_license': False, # "MIT"
    '--metadata_tags': False, # "sdvn.me"
}

def prompt(sample_prompt, sample_image_path):
  if "kontext" in model_type:
    if ControlFolder != "":
      s, p = random_sample(folder_train,ControlFolder)
    else:
      s, p = random_sample(folder_train,folder_train)
    if sample_prompt == "" :
      sample_prompt = s
    if sample_image_path == "":
      sample_image_path = p
    prompt = f"""{sample_prompt} --w {image_size(sample_image_path)[0]} --h {image_size(sample_image_path)[1]} --f 25 --ci {sample_image_path}"""
  else:
    prompt = f"""{sample_prompt if sample_prompt != "" else random_sample(folder_train)[0]} --w {sample_size[0]} --h {sample_size[1]} --f 25"""

  write_file(sample_prompt_path,prompt)

def dataset_file():
  dataset = os.path.join(root_dir,"dataset.toml")
  for dir in check_sub_dir(image_dir):
    data_config["datasets"].append({
        "image_directory": dir,
        "control_directory": dir.replace(ImageFolder, ControlFolder) if ControlFolder != "" else None,
        "cache_directory": os.path.join(root_dir,f"cache_{model_type}"),
        "num_repeats": repeat_dir(dir,num_repeats),
        "flux_kontext_no_resize_control": True if resize_control == False and "kontext" in model_type else None,
        })

  with open(dataset, "w") as file:
      toml.dump(data_config, file)

dataset_config_path = os.path.join(root_dir,"dataset.toml")

def cache_latents_config():
  return {
    "python": True,
    "flux_kontext_cache_latents.py": True if "kontext" in model_type else False,
    "qwen_image_cache_latents.py": True if "qwen" in model_type else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--vae": f'"{model_path(model_dic[model_type]["vae"])}"',
}

def cache_text_encoder_config():
  return {
    "python": True,
    "flux_kontext_cache_text_encoder_outputs.py": True if "kontext" in model_type else False,
    "qwen_image_cache_text_encoder_outputs.py": True if "qwen" in model_type else False,
    "--dataset_config": f'"{dataset_config_path}"',
    "--text_encoder": f'"{model_path(model_dic[model_type]["clip"])}"' if "clip2" not in model_dic[model_type] else False,
    "--text_encoder1": f'"{model_path(model_dic[model_type]["clip"])}"' if "clip2" in model_dic[model_type] else False,
    "--text_encoder2": f'"{model_path(model_dic[model_type]["clip2"])}"' if "clip2" in model_dic[model_type] else False,
    "--batch_size": 16
}